Import necessary packages.

In [59]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

Read in dataset as well as the tfidf csv we created in the last Jupyter Notebook.

In [60]:
df = pd.read_csv('hs_merged.csv', encoding = "ISO-8859-1", index_col=0)
tf = pd.read_csv('tfidf.csv', encoding = "ISO-8859-1", index_col=0)

Create lists of the sexist terms, racist terms and other terms. Then put those three lists into a list of lists ("classes"). Create a "bad list" of tfidf terms that we think are not predictive ("colin" is someone's name, for instance), and go through each list and remove those terms.

In [61]:
s_t = list(tf['Sexist_terms'])
r_t = list(tf['Racist_terms']) 
o_t = list(tf['Other_terms'])
classes = [s_t,r_t,o_t]

bad_list = ['colin','im','dont','ok','u','af','na','ur','p','^.^','lt','am','|']

for i in classes:
    for k in i:
        if k in bad_list:
            i.remove(k)

Create a list of tweets. For each list, and then for each term in that list, create an empty list. For each tweet in the dataset, check to see whether that tweet contains the term. If yes, add a 1 to the list. If not, add a 0 to the list. Add that list to the dataframe under a name that is the same as the term. To check whether the terms were added to your dataframe, uncomment the print statement at the end. You should see all tfidf terms listed.

In [73]:
tweets = list(df['Tweets']) 

for l in classes:
    for i in l:
        t = []
        for d in tweets:
            tfm = re.search(i, d)
            if tfm:
                t.append(1)
            else:
                t.append(0)
        df[i] = t    

    
names = df.columns.values 
#print(names)

We'll do something similar with hashtags, though we need to do a little preprocessing. Because this variable is not a simple list of hashtags, and also not (confusingly) a dictionary as it appears (to see what I'm talking about, uncomment the print statement above), we have to go through and extract the hashtags from the string for ourselves. Because it presents as a dictionary with keys, pull out all of the words and then get rid of the keywords "text" and "indices". Also convert to lowercase. Then create a set of only the unique values to use as variables in our dataset.

In [74]:
hashtags = list(df['Hashtags'])
htags = []

for hashtag in hashtags:
    texts = re.findall(r"'\w+'", hashtag)
    for i in texts:
        if i != 'text' and i != 'indices':
            htags.append(i.lower())

htags=set(htags)

Now that we have the actual hashtags, we have to go through the original hashtags variable and for each observation check whether a term in the hashtag list we created above is in the original variable. If it is, add a 1 to the list we create in our for loop. If not, add a zero. (This is very similar to what we did for our tfidf terms in the tweets).

In [75]:
list_h = list(df['Hashtags'])

for i in htags:
    list_i = []
    for k in list_h:
        ht = re.search(i,k)
        if ht:
            list_i.append(1)
        else:
            list_i.append(0)
    df[i] = list_i

The last new variable that we're adding is values for sentiment analysis. Create an analyzer object called "analyzer" with Vader's SentimentIntensityAnalyzer() function. Since hate speech is associated with negative valency, we've created two lists to hold the values for the negative sentiment and the compound (overall) sentiment. The for loop goes through all of the tweets in the tweets variable from the dataframe, which we've converted into a list, and assigns sentiment to each tweet. The values for the negative and compound sentiment are then appended to the empty lists we created for each, respectively.

In [76]:
tweets = list(df.Tweets)

analyzer = SentimentIntensityAnalyzer()
negative = []
compound = []

for i in range(len(tweets)):
    vs = analyzer.polarity_scores(tweets[i])
    negative.append(vs.get("neg"))
    compound.append(vs.get("compound"))

Add "Neg_Sent" and "Compound" variables to the dataframe, set each to the values from their lists above. To see the resulting dataframe, uncomment the print statement.

In [77]:
df['Neg_Sent'] = negative
df['Compound'] = compound

,sorri,celin
ID,,
319399851215433729,0,0
320817818222358529,0,0
324114200450437120,0,0
326286656854454273,0,0
381988216292655104,0,0


Save dataset with these new variables as a csv: this is our official final dataset that we will use in a neural network!

In [80]:
df.to_csv("full_dataset.csv")